### prueba scrapear!

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from fake_useragent import UserAgent

#### carrefour

In [2]:
ua = UserAgent()
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2226.0 Safari/537.36'}


In [16]:
list_pages_carrefour = list(range(0,336,24))
table_pages_carrefour=[]
for number in list_pages_carrefour:
    url_carrefour = f'https://www.carrefour.es/supermercado/bebidas/cerveza/todas-las-cervezas/N-1uq8b5u/c?No={number}'
    html = requests.get(url_carrefour, headers=header).text
    soup_carrefour = BeautifulSoup(html, 'lxml')
    table_carrefour = soup_carrefour.find_all({'article':'product-card-item'})
    table_pages_carrefour.append(table_carrefour)

In [17]:
table_pages_carrefour[0][24]

<article class="product-card-item">
<div class="load-more-products">
<p>SIGUIENTE PÁGINA</p>
<a class="pathCart" href="/supermercado/bebidas/cerveza/todas-las-cervezas/N-1uq8b5u/c?No=24&amp;Nr%3DAND%28product.shopCodes%3A004320%2Cproduct.salepointWithActivePrice_004320%3A1%2COR%28product.siteId%3AbasicSite%29%29OR%29">
</a>
</div>
</article>

In [18]:
len(table_pages_carrefour[0])

25

In [19]:
data_raw_carrefour=[]
for i in range(len(table_pages_carrefour)):
    for j in range(len(table_pages_carrefour[i])):
        rows_carrefour = table_pages_carrefour[i][j].find_all(['span','p','a'], {'class': ['price','price-less','format-price','js-gap-product-click-super']})
        promotion_carrefour = table_pages_carrefour[i][j].find_all(['p'], {'class': ['promocion-copy']})
        if len(rows_carrefour) != 0:
            if len(promotion_carrefour) != 0:
                data_raw_carrefour.append(rows_carrefour + promotion_carrefour)
            else:
                data_raw_carrefour.append(rows_carrefour)
                
            

In [20]:
data_raw_carrefour[20]

[<a class="js-gap-product-click-super" data-product-discount="0,07" data-productid="762603515" data-special-campaign="qvpromo730001_feliz-aniversario" href="/supermercado/cerveza-1906-reserva-especial-pack-de-12-botellas-33-cl-1906/R-762603515/p" title="Cerveza 1906 reserva especial pack de 12 botellas 33 cl.">
 <img alt="Cerveza 1906 reserva especial pack de 12 botellas 33 cl." aria-describedby="product-762603515" sizes="(max-width: 280px) 80px,(max-width: 490px) 100px, (max-width: 768px) 560px, (max-width: 991px) 280px, (max-width: 1280px) 560px" src="https://static.carrefour.es/hd_280x_/img_pim_food/771393_00_1.jpg" srcset="https://static.carrefour.es/hd_560x_/img_pim_food/771393_00_1.jpg 560w, https://static.carrefour.es/hd_280x_/img_pim_food/771393_00_1.jpg 280w, https://static.carrefour.es/hd_100x_/img_pim_food/771393_00_1.jpg 100w, https://static.carrefour.es/hd_100x_/img_pim_food/771393_00_1.jpg  80w" width="100%"/>
 </a>,
 <span class="price-less">
 11,40 €</span>,
 <p class="

In [21]:
len(data_raw_carrefour)

332

In [136]:
#price in euros
# price_raw = data_raw[0][1].text

def get_price(data_text):
    price_raw = data_text.text
    price = re.sub('\xa0€','',price_raw)
    price = re.sub('\n','',price)
    return float(re.sub(",",'.',price))

In [140]:
#price per liter
# price_l_raw = data_raw[0][2].text

def get_price_l(data_text):
    price_l_raw = data_text.text
    price_l = re.findall('[0-9]+,[0-9]+', price_l_raw)
    price_liter = re.sub(',','.',price_l[0])
    return float(price_liter)

In [24]:
#title
# title_raw = data_raw[0][3].text

def get_title(data_text):
    title_raw = data_text.text
    title = re.sub("\n",'',title_raw)
    return title

In [25]:
#promotion product

def get_promotion(data_text):
        promotion_raw = data_text.text
        promotion = re.sub("\n",'',promotion_raw)
        return promotion


In [423]:
#brand
def get_brand(data_text):
    brand_raw = data_text.text
    brand = re.split('(Cerveza )|( pack)|( botella)|( lata)+|(cl)',brand_raw)
    return brand[6]

In [27]:
#container
def get_container(data_text):
    container_raw = data_text.text
    container = re.findall('botella|lata|barril',container_raw)
    if len(container)!=0:
        return container[0]
    else:
        return 'No specified'

In [28]:
#volumen
def get_volumen_unid(data_text):
    volumen_raw = data_text.text 
    volumen = re.findall('[0-9]+ cl',volumen_raw)
    if len(volumen)!=0:
        return volumen[0]
    else:
        volumen_l = re.findall('[0-9]+[ mcl]+',volumen_raw)
        return volumen_l[0]

In [410]:
#quantity
def get_quantity_pack(data_text):
    quantity_raw = data_text.text
    quantity_split = re.split('pack de|pack',quantity_raw)
    if len(quantity_split)==2:
        quantity = re.findall('[0-9]+',quantity_split[1])
        return quantity[0]
    else:
        return 1

In [411]:
#image
def get_image(data_text):
    return data_text.find_all('img')[0].get('src')

In [424]:
data_beer_carrefour = pd.DataFrame(index=range(0,len(data_raw_carrefour)),columns=['price','price_liter',
                                                                           'title','promotion','brand',
                                                                          'container','volumen_unid',
                                                                          'quantity_pack','image_url',
                                                                        'supermarket'])
for beer_number in range(len(data_raw_carrefour)):
    data_beer_carrefour.iloc[beer_number,0] = get_price(data_raw_carrefour[beer_number][1])
    data_beer_carrefour.iloc[beer_number,1] = get_price_l(data_raw_carrefour[beer_number][2])
    data_beer_carrefour.iloc[beer_number,2] = get_title(data_raw_carrefour[beer_number][3])
    data_beer_carrefour.iloc[beer_number,4] = get_brand(data_raw_carrefour[beer_number][3])
    data_beer_carrefour.iloc[beer_number,5] = get_container(data_raw_carrefour[beer_number][3])
    data_beer_carrefour.iloc[beer_number,6] = get_volumen_unid(data_raw_carrefour[beer_number][3])
    data_beer_carrefour.iloc[beer_number,7] = get_quantity_pack(data_raw_carrefour[beer_number][3])
    data_beer_carrefour.iloc[beer_number,8] = get_image(data_raw_carrefour[beer_number][0])
    data_beer_carrefour.iloc[beer_number,9] = 'Carrefour'
    if len(data_raw_carrefour[beer_number]) > 4:
        data_beer_carrefour.iloc[beer_number,3] = get_promotion(data_raw_carrefour[beer_number][4])
    else:
        data_beer_carrefour.iloc[beer_number,3] = 'No promotion'


In [426]:
data_beer_carrefour.head()

,price,price_liter,title,promotion,brand,container,volumen_unid,quantity_pack,image_url,supermarket
0,11.21,1.89,Cerveza Heineken Lager pack de 18 latas de 33 cl.,No promotion,Heineken Lager,lata,33 cl,18,https://static.carrefour.es/hd_280x_/img_pim_f...,Carrefour
1,14.8,1.6,Cerveza Cruzcampo Pilsen pack de 28 latas de 3...,No promotion,Cruzcampo Pilsen,lata,33 cl,28,https://static.carrefour.es/hd_280x_/img_pim_f...,Carrefour
2,5.15,1.56,Cerveza El Aguila pack de 10 latas de 33 cl.,No promotion,El Aguila,lata,33 cl,10,https://static.carrefour.es/hd_280x_/img_pim_f...,Carrefour
3,0.25,0.76,Cerveza Carrefour Holandesa lata 33 cl.,No promotion,Carrefour Holandesa,lata,33 cl,1,https://static.carrefour.es/hd_280x_/img_pim_f...,Carrefour
4,11.7,1.95,Cerveza Estrella Galicia especial pack de 24 b...,No promotion,Estrella Galicia especial,botella,25 cl,24,https://static.carrefour.es/hd_280x_/img_pim_f...,Carrefour


In [139]:
sum(data_beer_carrefour['price'])

903.6500000000004

In [99]:
list_pages_alcampo = list(range(0,10))
table_pages_alcampo=[]
for number in list_pages_alcampo:
    url_alcampo = f'https://www.alcampo.es/compra-online/bebidas/cervezas/c/W1107?q=%3Arelevance&page={number}'
    html = requests.get(url_alcampo, headers=header).text
    soup_alcampo = BeautifulSoup(html, 'lxml')
    table_alcampo = soup_alcampo.find_all('div',{'class':'productGridItem'})
    table_pages_alcampo.append(table_alcampo)


In [100]:
len(table_pages_alcampo[0])

48

In [101]:
len(table_pages_alcampo)

10

In [102]:
data_raw_alcampo=[]
for i in range(len(table_pages_alcampo)):
    for j in range(len(table_pages_alcampo[i])):
        rows_alcampo = table_pages_alcampo[i][j].find_all(['span','div'],{'class': ['price',
                                                                                    'productName',
                                                                                    'thumb cut-alt-img']})
        promotion_alcampo = table_pages_alcampo[i][j].find_all('div', {'class': 'financiacionMensual'})
        if len(rows_alcampo) != 0:
            if len(promotion_alcampo) != 0:
                data_raw_alcampo.append(rows_alcampo + promotion_alcampo)
            else:
                data_raw_alcampo.append(rows_alcampo)
                

In [103]:
len(data_raw_alcampo)

430

In [165]:
#price alcampo
def get_price_alcampo(data_text):
    price_raw = data_text.text
    price = re.findall('[0-9,]+',price_raw)
    return float(re.sub(",",'.',price[0]))


In [175]:
#price_liter alcampo
def get_price_liter_alcampo(data_text):
    price_raw = data_text.text
    price = re.findall('[0-9,]+',price_raw)
    if len(price)==1:
        return float(re.sub(",",'.',price[0]))
    else:
        return float(re.sub(",",'.',price[1]))


In [187]:
#title alcampo
def get_tittle_alcampo(data_text):
    tittle_raw = data_text.text
    tittle = re.sub('\n','',tittle_raw)
    return tittle


In [394]:
#promotion alcampo
def get_promotion_alcampo(data_text):
    promotion_raw = data_text.text
    promotion = re.sub('\r\n\r\n\t\t\t\t\t\t\t\t\t\t','',promotion_raw)
    return promotion

In [422]:
#brand alcampo
def get_brand_alcampo(data_text):
    brand_raw = data_text.text
    brand = re.sub('\n','',brand_raw)
    brand_2 = re.findall('[A-Z 0-9,À-ÿ]{3,} ',brand)
    return brand_2[0]

In [396]:
#volumen alcampo
def get_volumen_unid_alcampo(data_text):
    volumen_raw = data_text.text 
    volumen = re.findall('[0-9]+ cl',volumen_raw)
    if len(volumen)!=0:
        return volumen[0]
    else:
        volumen_l = re.findall('[0-9]+[ mcl]+',volumen_raw)
        if len(volumen_l)==2:
            return volumen_l[1]
        else:
            return volumen_l[0]

In [436]:
#image alcampo
def get_image_alcampo(data_text):
    return data_text.find_all('img')[0].get('data-blzsrc')

In [442]:
data_raw_alcampo[0][1]

<div class="productName truncate">
<!-- Inicio UAT-203 -->
<span>Cerveza Clásica MAHOU pack de 28 latas de 33 cl.</span>
<!-- Fin UAT-203 -->
</div>

In [437]:
data_beer_alcampo= pd.DataFrame(index=range(0,len(data_raw_alcampo)),columns=['price','price_liter',
                                                                           'title','promotion','brand',
                                                                          'container','volumen_unid',
                                                                          'quantity_pack','image_url',
                                                                        'supermarket'])
for beer_number in range(len(data_raw_alcampo)):
    data_beer_alcampo.iloc[beer_number,0] = get_price_alcampo(data_raw_alcampo[beer_number][2])
    data_beer_alcampo.iloc[beer_number,1] = get_price_liter_alcampo(data_raw_alcampo[beer_number][2])
    data_beer_alcampo.iloc[beer_number,2] = get_tittle_alcampo(data_raw_alcampo[beer_number][1])
    data_beer_alcampo.iloc[beer_number,4] = get_brand_alcampo(data_raw_alcampo[beer_number][1])
    data_beer_alcampo.iloc[beer_number,5] = get_container(data_raw_alcampo[beer_number][1])
    data_beer_alcampo.iloc[beer_number,6] = get_volumen_unid_alcampo(data_raw_alcampo[beer_number][1])
    data_beer_alcampo.iloc[beer_number,7] = get_quantity_pack(data_raw_alcampo[beer_number][1])
    data_beer_alcampo.iloc[beer_number,8] = get_image_alcampo(data_raw_alcampo[beer_number][0])
    data_beer_alcampo.iloc[beer_number,9] = 'Alcampo'
    if len(data_raw_alcampo[beer_number]) > 3:
        data_beer_alcampo.iloc[beer_number,3] = get_promotion_alcampo(data_raw_alcampo[beer_number][3])
    else:
        data_beer_alcampo.iloc[beer_number,3] = 'No promotion'

In [439]:
data_beer_alcampo.head()

,price,price_liter,title,promotion,brand,container,volumen_unid,quantity_pack,image_url,supermarket
0,10.64,1.15,Cerveza Clásica MAHOU pack de 28 latas de 33 cl.,oferta válida hasta el 14/10/2020,MAHOU,lata,33 cl,28,https://www.alcampo.es/media/ha9/h39/100335357...,Alcampo
1,13.49,1.45,Cervezas MAHOU 5 ESTRELLAS pack 28 latas x 33 cl.,No promotion,MAHOU 5 ESTRELLAS,lata,33 cl,28,https://www.alcampo.es/media/h11/h90/902116140...,Alcampo
2,6.47,1.63,Cerveza HEINEKEN pack de 12 latas de 33 centi...,No promotion,HEINEKEN,lata,33 c,12,https://www.alcampo.es/media/h67/h4f/954684103...,Alcampo
3,4.54,1.14,"Cervezas AMSTEL 100 % MALTA pack 12 uds, de 3...",No promotion,AMSTEL 100,No specified,33 cl,12,https://www.alcampo.es/media/hbb/h21/104542976...,Alcampo
4,6.52,1.64,Cervezas Clásica MAHOU pack de 12 latas de 33...,No promotion,MAHOU,lata,33 c,12,https://www.alcampo.es/media/h64/hc7/965932407...,Alcampo
